In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tkinter import *
import tkinter as tk
from PIL import Image, ImageTk

In [2]:
class MinMaxScaler(object):
    def __init__(self, feature_range=(0, 1)):
        self.low_, self.high_ = feature_range

    def fit(self, X):
        self.min_ = X.min(axis=0)
        self.max_ = X.max(axis=0)
        return self

    def transform(self, X):
        X_std = (X - self.min_) / (self.max_ - self.min_)
        return X_std * (self.high_ - self.low_) + self.low_

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [3]:
def init_params():
    W1 = np.random.rand(8, 8) - 0.5
    b1 = np.random.rand(8, 1) - 0.5
    W2 = np.random.rand(6, 8) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    W3 = np.random.rand(1, 6) - 0.5
    b3 = np.random.rand(1, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(Z, 0)

def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = sigmoid(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    global m
    reshapeY = Y.reshape(618,1)
    dZ3 = A3 - reshapeY.T
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3
    
    return W1, b1, W2, b2, W3, b3

In [4]:
def get_predictions(A3):
    prediksi = []
    for a in A3[0]:
        if a >= 0.5:
            output = 1
        else:
            output = 0
        prediksi.append(output)
    return prediksi

def get_predictions_test(data):
    
    if data >= 0.5:
        output = 1
    else:
        output = 0
        
    return output

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    global akurasi
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            akurasi = get_accuracy(predictions, Y)
            print(akurasi)
    return W1, b1, W2, b2, W3, b3

In [5]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    
    if predictions == 1:
        diabetes = "Diabetes"
    else:
        diabetes = "Tidak Diabetes"
    
    return diabetes

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    global hasilprediksi_text, prediksi_btn, canvas

    prediction = make_predictions(index, W1, b1, W2, b2, W3, b3)
    
    canvas = tk.Canvas(root, width=600, height=300)
    canvas.grid(columnspan=8, rowspan=3)
    
    hasilprediksi_text = tk.Label(root, text="Prediksi : "+prediction, font=30)
    hasilprediksi_text.grid(columnspan=8, column=1, row=1)
    
    prediksi_btn = tk.Button(root, text="Prediksi", width=15, command=masukkan_prediksi)
    prediksi_btn.grid(columnspan=8, row=2, column=1)
    
    print(index)
    print("Prediction: ", prediction)

In [6]:
class Table:
    table = []
    def __init__(self,root, array, k):
        # code for creating table
        for i in range(total_rows):
            for j in range(total_columns):
                  
                self.e = Entry(root, width=20, fg='blue',
                               font=('Arial',8,'bold'))
                self.table.append(self.e)  
                self.e.grid(row=k, column=j)
                self.e.insert(END, array[i][j])
            k+=1
    def label_del(self) :
        for i in self.table :
            i.destroy()
            
def prediksi():
    global W1, W2, W3, b1, b2, b3
    global my_label_pregnant, my_box_pregnant, my_label_glucose, my_box_glucose, my_label_bloodpressure, my_box_bloodpressure
    global my_label_skinthickness, my_box_skinthickness, my_label_insulin, my_box_insulin, my_label_bmi, my_box_bmi
    global my_label_dpf, my_box_dpf, my_label_age, my_box_age, masukkan2_btn, canvas
    
    try:
        testarray = []
        
        pregnant = float(my_box_pregnant.get())
        glucose = float(my_box_glucose.get())
        bloodpressure = float(my_box_bloodpressure.get())
        skinthickness = float(my_box_skinthickness.get())
        insulin = float(my_box_insulin.get())
        bmi = float(my_box_bmi.get())
        dpf = float(my_box_dpf.get())
        age = float(my_box_age.get())
        
        testarray.append(pregnant)
        testarray.append(glucose)
        testarray.append(bloodpressure)
        testarray.append(skinthickness)
        testarray.append(insulin)
        testarray.append(bmi)
        testarray.append(dpf)
        testarray.append(age)
        
        my_label_pregnant.destroy()
        my_box_pregnant.destroy()
        my_label_glucose.destroy()
        my_box_glucose.destroy()
        my_label_bloodpressure.destroy()
        my_box_bloodpressure.destroy()
        my_label_skinthickness.destroy()
        my_box_skinthickness.destroy()
        my_label_insulin.destroy()
        my_box_insulin.destroy()
        my_label_bmi.destroy()
        my_box_bmi.destroy()
        my_label_dpf.destroy()
        my_box_dpf.destroy()
        my_label_age.destroy()
        my_box_age.destroy()
        masukkan2_btn.destroy()
        
        testarray = np.array(testarray)
        scaler = MinMaxScaler(feature_range=(0,1)).fit(testarray)
        testarray = scaler.transform(testarray)
        reshapetestarray = testarray.reshape(1,-1)
        
        canvas.destroy()
        
        test_prediction(reshapetestarray.T, W1, b1, W2, b2, W3, b3)
        
    except ValueError:
        my_label_pregnant.destroy()
        my_box_pregnant.destroy()
        my_label_glucose.destroy()
        my_box_glucose.destroy()
        my_label_bloodpressure.destroy()
        my_box_bloodpressure.destroy()
        my_label_skinthickness.destroy()
        my_box_skinthickness.destroy()
        my_label_insulin.destroy()
        my_box_insulin.destroy()
        my_label_bmi.destroy()
        my_box_bmi.destroy()
        my_label_dpf.destroy()
        my_box_dpf.destroy()
        my_label_age.destroy()
        my_box_age.destroy()
        masukkan2_btn.destroy()
        canvas.destroy()
        
        masukkan_prediksi()
    
def masukkan_prediksi():
    global W1_text, W2_text, W3_text, akurasi_text, t, t2, t3, prediksi_btn
    global my_label_pregnant, my_box_pregnant, my_label_glucose, my_box_glucose, my_label_bloodpressure, my_box_bloodpressure
    global my_label_skinthickness, my_box_skinthickness, my_label_insulin, my_box_insulin, my_label_bmi, my_box_bmi
    global my_label_dpf, my_box_dpf, my_label_age, my_box_age, masukkan2_btn, hasilprediksi_text, canvas
    
    canvas.destroy()
    hasilprediksi_text = tk.Label(root, text="Prediksi : ", font=30)
    
    W1_text.destroy()
    W2_text.destroy()
    W3_text.destroy()
    akurasi_text.destroy()
    t.label_del()
    t2.label_del()
    t3.label_del()
    prediksi_btn.destroy()
    hasilprediksi_text.destroy()
    canvas = tk.Canvas(root, width=600, height=300)
    canvas.grid(columnspan=8, rowspan=3)
    
    my_label_pregnant = Label(root, text="Pregnancies : ")
    my_label_pregnant.grid(column=1, row=0)

    my_box_pregnant = Entry(root)
    my_box_pregnant.grid(column=2, row=0)

    my_label_glucose = Label(root, text="Glucose : ")
    my_label_glucose.grid(column=3, row=0)

    my_box_glucose = Entry(root)
    my_box_glucose.grid(column=4, row=0)

    my_label_bloodpressure = Label(root, text="Blood Pressure : ")
    my_label_bloodpressure.grid(column=5, row=0)

    my_box_bloodpressure = Entry(root)
    my_box_bloodpressure.grid(column=6, row=0)

    my_label_skinthickness = Label(root, text="Skin Thickness : ")
    my_label_skinthickness.grid(column=7, row=0)

    my_box_skinthickness = Entry(root)
    my_box_skinthickness.grid(column=8, row=0)

    my_label_insulin = Label(root, text="Insulin : ")
    my_label_insulin.grid(column=1, row=1)

    my_box_insulin = Entry(root)
    my_box_insulin.grid(column=2, row=1)

    my_label_bmi = Label(root, text="BMI : ")
    my_label_bmi.grid(column=3, row=1)

    my_box_bmi = Entry(root)
    my_box_bmi.grid(column=4, row=1)

    my_label_dpf = Label(root, text="Diabetes Predigree Function : ")
    my_label_dpf.grid(column=5, row=1)

    my_box_dpf = Entry(root)
    my_box_dpf.grid(column=6, row=1)

    my_label_age = Label(root, text="Age : ")
    my_label_age.grid(column=7, row=1)

    my_box_age = Entry(root)
    my_box_age.grid(column=8, row=1)
    
    masukkan2_btn = tk.Button(root, text="Mulai Prediksi", width=15, command=prediksi)
    masukkan2_btn.grid(columnspan=8, row=2, column=1)

def mulai():
    global m, total_rows, total_columns, akurasi, W1_text, W2_text, W3_text, my_label_learningrate, my_label_iterasi, canvas
    global akurasi_text, t, t2, t3, W1, W2, W3, b1, b2, b3, my_box_learningrate, my_box_iterasi, masukkan_btn, prediksi_btn
    
    try:
        learningrate = float(my_box_learningrate.get())
        iterasi = int(my_box_iterasi.get())
        
        my_label_learningrate.destroy()
        my_label_iterasi.destroy()
        my_box_learningrate.destroy()
        my_box_iterasi.destroy()
        masukkan_btn.destroy()

        data = pd.read_csv('diabetes.csv')

        data = np.array(data)
        m, n = data.shape
        np.random.shuffle(data)

        data_dev = data[0:150].T
        Y_dev = data_dev[-1]
        X_dev = data_dev[:n-1]

        data_train = data[150:m].T
        Y_train = data_train[-1]
        X_train = data_train[:n-1]

        scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)
        X_train = scaler.transform(X_train)

        W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, learningrate, iterasi)

        total_rows = len(W1)
        total_columns = len(W1[0])

        W1_text = tk.Label(root, text="W1 hasil training", font=30)
        W1_text.grid(columnspan=8, column=1, row=1)
        
        t = Table(root, W1, 2)

        total_rows = len(W2)
        total_columns = len(W2[0])

        W2_text = tk.Label(root, text="W2 hasil training", font=30)
        W2_text.grid(columnspan=8, column=1, row=10)

        t2 = Table(root, W2, 11)

        total_rows = len(W3)
        total_columns = len(W3[0])

        W3_text = tk.Label(root, text="W3 hasil training", font=30)
        W3_text.grid(columnspan=8, column=1, row=17)

        t3 = Table(root, W3, 18)

        akurasi = str(akurasi)

        akurasi_text = tk.Label(root, text="Accuracy : "+akurasi, font=30)
        akurasi_text.grid(columnspan=8, column=1, row=19)
        
        canvas.destroy()
        
        prediksi_btn = tk.Button(root, text="Prediksi", width=15, command=masukkan_prediksi)
        prediksi_btn.grid(columnspan=8, row=20, column=1)
    
    except ValueError:
        my_label_learningrate.destroy()
        my_label_iterasi.destroy()
        my_box_learningrate.destroy()
        my_box_iterasi.destroy()
        masukkan_btn.destroy()
        masukkan()

def masukkan():
    global judul_text, logo_label, start_btn, my_box_learningrate, my_box_iterasi, my_label_learningrate, my_label_iterasi
    global masukkan_btn, canvas
    
    judul_text.destroy()
    logo_label.destroy()
    start_btn.destroy()
    
    my_label_learningrate = Label(root, text="Learning Rate : ")
    my_label_learningrate.grid(column=1, row=0)

    my_box_learningrate = Entry(root)
    my_box_learningrate.grid(column=2, row=0)

    my_label_iterasi = Label(root, text="Jumlah Iterasi : ")
    my_label_iterasi.grid(column=3, row=0)

    my_box_iterasi = Entry(root)
    my_box_iterasi.grid(column=4, row=0)
    
    masukkan_btn = tk.Button(root, text="Mulai Proses Training", width=18, command=mulai)
    masukkan_btn.grid(columnspan=8, row=1, column=1)

global judul_text, logo_label, start_btn, canvas
    
root = tk.Tk()
canvas = tk.Canvas(root, width=600, height=300)
canvas.grid(columnspan=8, rowspan=3)

logo = Image.open('logo.jpg')
logo = logo.resize((200, 200), Image.ANTIALIAS)
logo = ImageTk.PhotoImage(logo)
logo_label = tk.Label(image=logo)
logo_label.image = logo
logo_label.grid(columnspan=8, row=1, column=1)

judul_text = tk.Label(root, text="Prediksi Diabetes Menggunakan Jaringan Syaraf Tiruan", font=30)
judul_text.grid(columnspan=8, column=1, row=0)

start_btn = tk.Button(root, text="Mulai", width=15, command=masukkan)
start_btn.grid(columnspan=8, row=2, column=1)

root.mainloop()

Iteration:  0
0.6472491909385113
Iteration:  10
0.6472491909385113
Iteration:  20
0.6472491909385113
Iteration:  30
0.6472491909385113
Iteration:  40
0.6456310679611651
Iteration:  50
0.6504854368932039
Iteration:  60
0.6521035598705501
Iteration:  70
0.6537216828478964
Iteration:  80
0.6666666666666666
Iteration:  90
0.6715210355987055
Iteration:  100
0.6715210355987055
Iteration:  110
0.6763754045307443
Iteration:  120
0.6747572815533981
Iteration:  130
0.6796116504854369
Iteration:  140
0.6779935275080906
Iteration:  150
0.6844660194174758
Iteration:  160
0.6844660194174758
Iteration:  170
0.6812297734627831
Iteration:  180
0.6844660194174758
Iteration:  190
0.6877022653721683
Iteration:  200
0.686084142394822
Iteration:  210
0.6974110032362459
Iteration:  220
0.6893203883495146
Iteration:  230
0.6925566343042071
Iteration:  240
0.6925566343042071
Iteration:  250
0.6893203883495146
Iteration:  260
0.7006472491909385
Iteration:  270
0.6909385113268608
Iteration:  280
0.69902912621359